# DATA INGESTION 

In [129]:
import boto3
s3_client = boto3.client('s3')
bucket_name= 'aws-a9097-use1-00-d-s3b-shrd-shr-data01'
IAM_ROLE='aws-a9097-use1-00-d-rol-shrd-shr-usr01'
check_response= s3_client.list_buckets();
print(check_response)

{'ResponseMetadata': {'RequestId': 'PP9JNN180J1KB0TY', 'HostId': 'xEkh8KozQOpI+0HXedpKo4TLEdAAlV/sqO/2z1uxt40t+7tXij8ZLH9vUvKdxWwx9N+Gx6+jiXM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'xEkh8KozQOpI+0HXedpKo4TLEdAAlV/sqO/2z1uxt40t+7tXij8ZLH9vUvKdxWwx9N+Gx6+jiXM=', 'x-amz-request-id': 'PP9JNN180J1KB0TY', 'date': 'Tue, 18 Feb 2025 03:52:48 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'aws-a9097-use1-00-d-s3b-shrd-shr-data01', 'CreationDate': datetime.datetime(2024, 8, 6, 8, 41, 20, tzinfo=tzlocal())}, {'Name': 'cf-templates-1jlshq8udppq6-us-east-1', 'CreationDate': datetime.datetime(2024, 8, 1, 10, 43, 34, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'aws.a9097', 'ID': '6d9499ce97be2b4c12d262e50a6c0e45aa4f21142385c704f4dffe905958950b'}}


In [131]:
files_list= ['GeoHiery.txt' , 'MarketDefinition.txt' , 'Prescriber.txt' , 'PrescriberAddress.txt' , 'ProductMaster.txt' , 'Sales.txt' , 'Zip_To_Terr.txt']

#Files to S3 
for file in files_list:
    s3_client.upload_file(file, bucket_name , f'raw/{file}')

# EXTRACTION

In [158]:
import pandas as pd


for file in files_list:
    s3_file= f's3://{bucket_name}/raw/{file}'
    df = pd.read_csv(s3_file, delimiter='|')
    print(df.head())

   GEO_CODE    GEO_NAME GEO_LEVEL PARENT_ID
0  T-111100      Acampo      TERR  D-111000
1  D-111000  District 1      DIST  R-110000
2  R-110000          CA      REGN  N-100000
3  N-100000         USA    NATION         -
4  T-111200       Acton      TERR  D-111000
  MKT_CD          MKT_NM
0  M-001    NEUROSCIENCE
1  M-002        VIROLOGY
2  M-003      IMMUNOLOGY
3  M-004      HEMATOLOGY
4  M-005  CARDIOVASCULAR
  Prescriber_ID FIRST_NAME  LAST_NAME         PREFFERED_NAME  GENDER  EMAIL  \
0      ST_00001   SOONTORN  THRUPKAEW     SOONTORN THRUPKAEW     NaN    NaN   
1      RB_00002     ROBERT      BRYAN           ROBERT BRYAN     NaN    NaN   
2      JC_00003       JOHN     CARTER  JOHN JEFFERSON CARTER     NaN    NaN   
3      TK_00004   THEODORE   KIRKLAND    THEODORE N KIRKLAND     NaN    NaN   
4      TC_00005        TOM   CALDWELL           TOM CALDWELL     NaN    NaN   

   PERSON_SPECIALTY  
0  GastroEnterology  
1         NeuroLogy  
2       Dermatology  
3         NeuroLogy  
4

REDSHIFT CONNECTION

In [262]:
import psycopg2

# REDSHIFT CREDS
REDSHIFT_HOST = 'aws-a9097-use1-00-d-red-shrd-shr-red01.ca3wdeu8uqbl.us-east-1.redshift.amazonaws.com'
REDSHIFT_DB = 'dev'
REDSHIFT_USER = 'group1user'
REDSHIFT_PORT = '5439'
REDSHIFT_PASSWORD = 'Group1pass'
conn = psycopg2.connect(dbname= REDSHIFT_DB,
                       user=REDSHIFT_USER,
                       password=REDSHIFT_PASSWORD,
                       host=REDSHIFT_HOST,
                        port=REDSHIFT_PORT
                       )
cursor = conn.cursor();
cursor.execute("SELECT current_date")
print('REDSHIFT CREDS LOADED SUCESSFULLY')

REDSHIFT CREDS LOADED SUCESSFULLY


# CREATE AND COPY TO BASE

In [170]:
sql1 = '''CREATE TABLE IF NOT EXISTS bs_grp1_person_profile(
    Prescriber_ID VARCHAR(50),
    FIRST_NAME VARCHAR(100),
    LAST_NAME VARCHAR(100),
    PREFFERED_NAME VARCHAR(100),
    GENDER VARCHAR(10),
    EMAIL VARCHAR(100),
    PERSON_SPECIALTY VARCHAR(100)
);'''

# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql1)
conn.commit()


In [171]:
sql2 = '''
COPY bs_grp1_person_profile
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Prescriber.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642'
;
'''


# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql2)
conn.commit()

In [172]:
sql_create = '''CREATE TABLE IF NOT EXISTS bs_grp1_prod_master(
    MARKET_CD VARCHAR(10),
    PRD_PACK_ID VARCHAR(50),
    PRODUCT_NAME VARCHAR(200),
    IMS_STRENG_TH_LEVEL VARCHAR(50),
    BRAND_ID VARCHAR(50),
    BRAND_NAME VARCHAR(100),
    PROD_MNF VARCHAR(100)
);'''

# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_create)
conn.commit()
print("Successfully created prod_master")

sql_copy = '''
COPY bs_grp1_prod_master
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/ProductMaster.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642'
;
'''


# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_copy)
conn.commit()
print("Data loaded in table")


Successfully created prod_master
Data loaded in table


In [173]:
#create_person_address
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_person_address (
    PERSON_CODE VARCHAR(50),
    ADDRESS_1 VARCHAR(200),
    ADDRESS_2 VARCHAR(200),
    CITY VARCHAR(100),
    STATE VARCHAR(50),
    ZIP_CODE VARCHAR(10)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_person_address successfully!")

# COPY FROM S3
sql_copy = """
COPY bs_grp1_person_address
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/PrescriberAddress.txt'
DELIMITER '|'
IGNOREHEADER 1
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_person_address from S3!")



Created table bs_grp1_person_address successfully!
Data loaded into bs_grp1_person_address from S3!


In [174]:
# CREATE TABLE_geo_hierachy
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_geo_hierarchy (
    GEO_CODE   VARCHAR(50),
    GEO_NAME   VARCHAR(100),
    GEO_LEVEL  VARCHAR(20),
    PARENT_ID  VARCHAR(50)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_geo_hierarchy successfully!")
# COPY
sql_copy = """
COPY bs_grp1_geo_hierarchy
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/GeoHiery.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_geo_hierarchy from S3!")




Created table bs_grp1_geo_hierarchy successfully!
Data loaded into bs_grp1_geo_hierarchy from S3!


In [175]:
# CREATE TABLE
sql_create3 = """
CREATE TABLE IF NOT EXISTS bs_grp1_zip_terr (
    ZIP_Code      VARCHAR(10),
    Territory_ID  VARCHAR(50)
);
"""
conn.rollback()
cursor.execute(sql_create3)
conn.commit()
print("Created table bs_grp1_zip_terr successfully!")
# COPY
sql_copy3 = """
COPY bs_grp1_zip_terr
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Zip_To_Terr.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy3)
conn.commit()
print("Data loaded into bs_grp1_zip_terr from S3!")


Created table bs_grp1_zip_terr successfully!
Data loaded into bs_grp1_zip_terr from S3!


In [176]:

sql_create2 = """
CREATE TABLE IF NOT EXISTS bs_grp1_sales (
    PSBR_ID          VARCHAR(50),
    PROD_ID          VARCHAR(50),
    MONTH_1_NRX_UNIT VARCHAR(50),
    MONTH_2_NRX_UNIT VARCHAR(50),
    MONTH_3_NRX_UNIT VARCHAR(50),
    MONTH_4_NRX_UNIT VARCHAR(50),
    MONTH_5_NRX_UNIT VARCHAR(50),
    MONTH_6_NRX_UNIT VARCHAR(50),
    MONTH_7_NRX_UNIT VARCHAR(50),
    MONTH_8_NRX_UNIT VARCHAR(50),
    MONTH_9_NRX_UNIT VARCHAR(50),
    MONTH_10_NRX_UNIT VARCHAR(50),
    MONTH_11_NRX_UNIT VARCHAR(50),
    MONTH_12_NRX_UNIT VARCHAR(50),
    MONTH_1_NRX_VOL VARCHAR(50),
    MONTH_2_NRX_VOL VARCHAR(50),
    MONTH_3_NRX_VOL VARCHAR(50),
    MONTH_4_NRX_VOL VARCHAR(50),
    MONTH_5_NRX_VOL VARCHAR(50),
    MONTH_6_NRX_VOL VARCHAR(50),
    MONTH_7_NRX_VOL VARCHAR(50),
    MONTH_8_NRX_VOL VARCHAR(50),
    MONTH_9_NRX_VOL VARCHAR(50),
    MONTH_10_NRX_VOL VARCHAR(50),
    MONTH_11_NRX_VOL VARCHAR(50),
    MONTH_12_NRX_VOL VARCHAR(50),
    MONTH_1_NRX_DOLLAR VARCHAR(50),
    MONTH_2_NRX_DOLLAR VARCHAR(50),
    MONTH_3_NRX_DOLLAR VARCHAR(50),
    MONTH_4_NRX_DOLLAR VARCHAR(50),
    MONTH_5_NRX_DOLLAR VARCHAR(50),
    MONTH_6_NRX_DOLLAR VARCHAR(50),
    MONTH_7_NRX_DOLLAR VARCHAR(50),
    MONTH_8_NRX_DOLLAR VARCHAR(50),
    MONTH_9_NRX_DOLLAR VARCHAR(50),
    MONTH_10_NRX_DOLLAR VARCHAR(50),
    MONTH_11_NRX_DOLLAR VARCHAR(50),
    MONTH_12_NRX_DOLLAR VARCHAR(50),
    MONTH_1_RRX_UNIT VARCHAR(50),
    MONTH_2_RRX_UNIT VARCHAR(50),
    MONTH_3_RRX_UNIT VARCHAR(50),
    MONTH_4_RRX_UNIT VARCHAR(50),
    MONTH_5_RRX_UNIT VARCHAR(50),
    MONTH_6_RRX_UNIT VARCHAR(50),
    MONTH_7_RRX_UNIT VARCHAR(50),
    MONTH_8_RRX_UNIT VARCHAR(50),
    MONTH_9_RRX_UNIT VARCHAR(50),
    MONTH_10_RRX_UNIT VARCHAR(50),
    MONTH_11_RRX_UNIT VARCHAR(50),
    MONTH_12_RRX_UNIT VARCHAR(50),
    MONTH_1_RRX_VOL VARCHAR(50),
    MONTH_2_RRX_VOL VARCHAR(50),
    MONTH_3_RRX_VOL VARCHAR(50),
    MONTH_4_RRX_VOL VARCHAR(50),
    MONTH_5_RRX_VOL VARCHAR(50),
    MONTH_6_RRX_VOL VARCHAR(50),
    MONTH_7_RRX_VOL VARCHAR(50),
    MONTH_8_RRX_VOL VARCHAR(50),
    MONTH_9_RRX_VOL VARCHAR(50),
    MONTH_10_RRX_VOL VARCHAR(50),
    MONTH_11_RRX_VOL VARCHAR(50),
    MONTH_12_RRX_VOL VARCHAR(50),
    MONTH_1_RRX_DOLLAR VARCHAR(50),
    MONTH_2_RRX_DOLLAR VARCHAR(50),
    MONTH_3_RRX_DOLLAR VARCHAR(50),
    MONTH_4_RRX_DOLLAR VARCHAR(50),
    MONTH_5_RRX_DOLLAR VARCHAR(50),
    MONTH_6_RRX_DOLLAR VARCHAR(50),
    MONTH_7_RRX_DOLLAR VARCHAR(50),
    MONTH_8_RRX_DOLLAR VARCHAR(50),
    MONTH_9_RRX_DOLLAR VARCHAR(50),
    MONTH_10_RRX_DOLLAR VARCHAR(50),
    MONTH_11_RRX_DOLLAR VARCHAR(50),
    MONTH_12_RRX_DOLLAR VARCHAR(50),
    MONTH_1_TRX_UNIT VARCHAR(50),
    MONTH_2_TRX_UNIT VARCHAR(50),
    MONTH_3_TRX_UNIT VARCHAR(50),
    MONTH_4_TRX_UNIT VARCHAR(50),
    MONTH_5_TRX_UNIT VARCHAR(50),
    MONTH_6_TRX_UNIT VARCHAR(50),
    MONTH_7_TRX_UNIT VARCHAR(50),
    MONTH_8_TRX_UNIT VARCHAR(50),
    MONTH_9_TRX_UNIT VARCHAR(50),
    MONTH_10_TRX_UNIT VARCHAR(50),
    MONTH_11_TRX_UNIT VARCHAR(50),
    MONTH_12_TRX_UNIT VARCHAR(50),
    MONTH_1_TRX_VOL VARCHAR(50),
    MONTH_2_TRX_VOL VARCHAR(50),
    MONTH_3_TRX_VOL VARCHAR(50),
    MONTH_4_TRX_VOL VARCHAR(50),
    MONTH_5_TRX_VOL VARCHAR(50),
    MONTH_6_TRX_VOL VARCHAR(50),
    MONTH_7_TRX_VOL VARCHAR(50),
    MONTH_8_TRX_VOL VARCHAR(50),
    MONTH_9_TRX_VOL VARCHAR(50),
    MONTH_10_TRX_VOL VARCHAR(50),
    MONTH_11_TRX_VOL VARCHAR(50),
    MONTH_12_TRX_VOL VARCHAR(50),
    MONTH_1_TRX_DOLLAR VARCHAR(50),
    MONTH_2_TRX_DOLLAR VARCHAR(50),
    MONTH_3_TRX_DOLLAR VARCHAR(50),
    MONTH_4_TRX_DOLLAR VARCHAR(50),
    MONTH_5_TRX_DOLLAR VARCHAR(50),
    MONTH_6_TRX_DOLLAR VARCHAR(50),
    MONTH_7_TRX_DOLLAR VARCHAR(50),
    MONTH_8_TRX_DOLLAR VARCHAR(50),
    MONTH_9_TRX_DOLLAR VARCHAR(50),
    MONTH_10_TRX_DOLLAR VARCHAR(50),
    MONTH_11_TRX_DOLLAR VARCHAR(50),
    MONTH_12_TRX_DOLLAR VARCHAR(50)

);
"""

conn.rollback()
cursor.execute(sql_create2)
conn.commit()
print("Created table bs_grp1_sales successfully!")
# COPY
sql_copy2 = """
COPY bs_grp1_sales
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Sales.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy2)
conn.commit()
print("Data loaded into bs_grp1_sales from S3!")


Created table bs_grp1_sales successfully!
Data loaded into bs_grp1_sales from S3!


In [177]:
# CREATE THE TABLE
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_mkt_desc (
    MKT_CD  VARCHAR(10),
    MKT_NM  VARCHAR(200)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_mkt_desc successfully!")

# COPY THE TABLE
sql_copy = """
COPY bs_grp1_mkt_desc
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/MarketDefinition.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_mkt_desc from S3!")

Created table bs_grp1_mkt_desc successfully!
Data loaded into bs_grp1_mkt_desc from S3!


In [178]:
sql_create = '''CREATE TABLE IF NOT EXISTS bs_grp1_prod_master(
    MARKET_CD VARCHAR(10),
    PRD_PACK_ID VARCHAR(50),
    PRODUCT_NAME VARCHAR(200),
    IMS_STRENG_TH_LEVEL VARCHAR(50),
    BRAND_ID VARCHAR(50),
    BRAND_NAME VARCHAR(100),
    PROD_MNF VARCHAR(100)
);'''

# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_create)
conn.commit()
print("Successfully created prod_master")

sql_copy = '''
COPY bs_grp1_prod_master
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Prescriber.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642'
;
'''


# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_copy)
conn.commit()
print("Data loaded in table")


Successfully created prod_master
Data loaded in table


In [179]:
import pandas as pd

df = pd.read_csv('Sales.txt', sep="|")
col1='PSBR_ID'
# col2='PROD_ID'

duplicates=df.duplicated(subset=[col1] , keep=False)
duplicate_count=duplicates.sum()

is_unique = duplicates.sum()

if is_unique:
    print("uq")
else:
    print("nah")

nah


# CREATING STAGING TABLES

Prescriber (stg_grp1_prescriber)

In [187]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_prescriber (
    PRESCRIBER_ID VARCHAR(255) NOT NULL UNIQUE,
    FIRST_NAME VARCHAR(255),
    LAST_NAME VARCHAR(255),
    PREFFERED_NAME VARCHAR(255),
    GENDER VARCHAR(255),
    EMAIL VARCHAR(255),
    PERSON_SPECIALTY VARCHAR(255),
    PRIMARY KEY (PRESCRIBER_ID)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_prescriber successfully!")

Created table stg_grp1_prescriber successfully!


Zip_Terr (stg_grp1_zip_terr)

In [247]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_zip_terr (
    ZIP_CODE VARCHAR(255) NOT NULL,
    Territory_ID VARCHAR(255) UNIQUE,
    PRIMARY KEY (ZIP_CODE)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_zip_terr successfully!")


Created table stg_grp1_zip_terr successfully!


Prescriber_Address (stg_grp1_prescriber_address)

In [248]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_prescriber_address (
    PRESCRIBER_ID VARCHAR(255) NOT NULL UNIQUE,
    ADDRESS_1 VARCHAR(255),
    ADDRESS_2 VARCHAR(255),
    CITY VARCHAR(255),
    STATE VARCHAR(255),
    ZIP_CODE VARCHAR(255),
    PRIMARY KEY (PRESCRIBER_ID),
    FOREIGN KEY (ZIP_CODE) REFERENCES stg_grp1_zip_terr(ZIP_CODE),
    FOREIGN KEY (PRESCRIBER_ID) REFERENCES stg_grp1_prescriber(PRESCRIBER_ID)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_prescriber_address successfully!")


Created table stg_grp1_prescriber_address successfully!


Mkt_Desc (stg_grp1_mkt_desc)

In [192]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_mkt_desc (
    MKT_CD VARCHAR(255) NOT NULL UNIQUE,
    MKT_NM VARCHAR(255),
    PRIMARY KEY (MKT_CD)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_mkt_desc successfully!")


Created table stg_grp1_mkt_desc successfully!


ProdMaster (stg_grp1_prod_master)

In [193]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_prod_master (
    MKT_CD VARCHAR(255),
    PRD_PACK_ID INT NOT NULL UNIQUE,
    PRODUCT_NAME VARCHAR(255),
    STRENGTH VARCHAR(255),
    BRAND_ID INT,
    Brand_Name VARCHAR(255),
    PROD_MNF VARCHAR(255),
    PRIMARY KEY (PRD_PACK_ID),
    FOREIGN KEY (MKT_CD) REFERENCES stg_grp1_mkt_desc(MKT_CD)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_prod_master successfully!")


Created table stg_grp1_prod_master successfully!


Sales (stg_grp1_sales)

In [194]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_sales (
    PRESCRIBER_ID VARCHAR(255) NOT NULL,
    PRD_PACK_ID INT NOT NULL,
    MONTH_1_NRX_UNIT INT,
    MONTH_2_NRX_UNIT INT,
    MONTH_3_NRX_UNIT INT,
    MONTH_4_NRX_UNIT INT,
    MONTH_5_NRX_UNIT INT,
    MONTH_6_NRX_UNIT INT,
    MONTH_7_NRX_UNIT INT,
    MONTH_8_NRX_UNIT INT,
    MONTH_9_NRX_UNIT INT,
    MONTH_10_NRX_UNIT INT,
    MONTH_11_NRX_UNIT INT,
    MONTH_12_NRX_UNIT INT,
    MONTH_1_NRX_VOL INT,
    MONTH_2_NRX_VOL INT,
    MONTH_3_NRX_VOL INT,
    MONTH_4_NRX_VOL INT,
    MONTH_5_NRX_VOL INT,
    MONTH_6_NRX_VOL INT,
    MONTH_7_NRX_VOL INT,
    MONTH_8_NRX_VOL INT,
    MONTH_9_NRX_VOL INT,
    MONTH_10_NRX_VOL INT,
    MONTH_11_NRX_VOL INT,
    MONTH_12_NRX_VOL INT,
    MONTH_1_NRX_DOLLAR INT,
    MONTH_2_NRX_DOLLAR INT,
    MONTH_3_NRX_DOLLAR INT,
    MONTH_4_NRX_DOLLAR INT,
    MONTH_5_NRX_DOLLAR INT,
    MONTH_6_NRX_DOLLAR INT,
    MONTH_7_NRX_DOLLAR INT,
    MONTH_8_NRX_DOLLAR INT,
    MONTH_9_NRX_DOLLAR INT,
    MONTH_10_NRX_DOLLAR INT,
    MONTH_11_NRX_DOLLAR INT,
    MONTH_12_NRX_DOLLAR INT,
    MONTH_1_RRX_UNIT INT,
    MONTH_2_RRX_UNIT INT,
    MONTH_3_RRX_UNIT INT,
    MONTH_4_RRX_UNIT INT,
    MONTH_5_RRX_UNIT INT,
    MONTH_6_RRX_UNIT INT,
    MONTH_7_RRX_UNIT INT,
    MONTH_8_RRX_UNIT INT,
    MONTH_9_RRX_UNIT INT,
    MONTH_10_RRX_UNIT INT,
    MONTH_11_RRX_UNIT INT,
    MONTH_12_RRX_UNIT INT,
    MONTH_1_RRX_VOL INT,
    MONTH_2_RRX_VOL INT,
    MONTH_3_RRX_VOL INT,
    MONTH_4_RRX_VOL INT,
    MONTH_5_RRX_VOL INT,
    MONTH_6_RRX_VOL INT,
    MONTH_7_RRX_VOL INT,
    MONTH_8_RRX_VOL INT,
    MONTH_9_RRX_VOL INT,
    MONTH_10_RRX_VOL INT,
    MONTH_11_RRX_VOL INT,
    MONTH_12_RRX_VOL INT,
    MONTH_1_RRX_DOLLAR INT,
    MONTH_2_RRX_DOLLAR INT,
    MONTH_3_RRX_DOLLAR INT,
    MONTH_4_RRX_DOLLAR INT,
    MONTH_5_RRX_DOLLAR INT,
    MONTH_6_RRX_DOLLAR INT,
    MONTH_7_RRX_DOLLAR INT,
    MONTH_8_RRX_DOLLAR INT,
    MONTH_9_RRX_DOLLAR INT,
    MONTH_10_RRX_DOLLAR INT,
    MONTH_11_RRX_DOLLAR INT,
    MONTH_12_RRX_DOLLAR INT,
    MONTH_1_TRX_UNIT INT,
    MONTH_2_TRX_UNIT INT,
    MONTH_3_TRX_UNIT INT,
    MONTH_4_TRX_UNIT INT,
    MONTH_5_TRX_UNIT INT,
    MONTH_6_TRX_UNIT INT,
    MONTH_7_TRX_UNIT INT,
    MONTH_8_TRX_UNIT INT,
    MONTH_9_TRX_UNIT INT,
    MONTH_10_TRX_UNIT INT,
    MONTH_11_TRX_UNIT INT,
    MONTH_12_TRX_UNIT INT,
    MONTH_1_TRX_VOL INT,
    MONTH_2_TRX_VOL INT,
    MONTH_3_TRX_VOL INT,
    MONTH_4_TRX_VOL INT,
    MONTH_5_TRX_VOL INT,
    MONTH_6_TRX_VOL INT,
    MONTH_7_TRX_VOL INT,
    MONTH_8_TRX_VOL INT, 
    MONTH_9_TRX_VOL INT,
    MONTH_10_TRX_VOL INT,
    MONTH_11_TRX_VOL INT,
    MONTH_12_TRX_VOL INT,
    MONTH_1_TRX_DOLLAR INT,
    MONTH_2_TRX_DOLLAR INT,
    MONTH_3_TRX_DOLLAR INT,
    MONTH_4_TRX_DOLLAR INT,
    MONTH_5_TRX_DOLLAR INT,
    MONTH_6_TRX_DOLLAR INT,
    MONTH_7_TRX_DOLLAR INT,
    MONTH_8_TRX_DOLLAR INT,
    MONTH_9_TRX_DOLLAR INT,
    MONTH_10_TRX_DOLLAR INT,
    MONTH_11_TRX_DOLLAR INT,
    MONTH_12_TRX_DOLLAR INT,
    PRIMARY KEY (PRESCRIBER_ID, PRD_PACK_ID),
    FOREIGN KEY (PRESCRIBER_ID) REFERENCES stg_grp1_prescriber(PRESCRIBER_ID),
    FOREIGN KEY (PRD_PACK_ID) REFERENCES stg_grp1_prod_master(PRD_PACK_ID)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_sales successfully!")

Created table stg_grp1_sales successfully!


Geo_Hierarchy (stg_grp1_geo_hierarchy)

In [197]:
sql_create = """
CREATE TABLE IF NOT EXISTS stg_grp1_geo_hierarchy (
    GEO_CODE VARCHAR(255) NOT NULL UNIQUE,
    GEO_NAME VARCHAR(255),
    GEO_LEVEL VARCHAR(255),
    PARENT_ID VARCHAR(255),
    PRIMARY KEY (GEO_CODE),
    FOREIGN KEY (GEO_CODE) REFERENCES stg_grp1_zip_terr(territory_id)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table stg_grp1_geo_hierarchy successfully!")


Created table stg_grp1_geo_hierarchy successfully!


#  DATA QUALITY MANAGMENT CHECKS

PK Check , Duplicate , Null check 

In [203]:
# Check for Primary Key consistency (i.e., unique and non-null PRESCRIBER_ID)
conn.rollback()

cursor.execute("""
    SELECT COUNT(*) 
    FROM bs_grp1_person_profile 
    WHERE PRESCRIBER_ID IS NULL;
""")
null_count = cursor.fetchone()[0]
if null_count > 0:
    print(f"Warning: There are {null_count} NULL values in PRESCRIBER_ID column!")

conn.rollback()
 
cursor.execute("""
    SELECT COUNT(DISTINCT PRESCRIBER_ID) 
    FROM bs_grp1_person_profile;
""")
distinct_count = cursor.fetchone()[0]
conn.rollback()

cursor.execute("""
    SELECT COUNT(PRESCRIBER_ID) 
    FROM bs_grp1_person_profile;
""")
total_count = cursor.fetchone()[0]

if distinct_count != total_count:
    print(f"Warning: There are duplicates in the PRESCRIBER_ID column!")

# Check for duplicates in other columns (First Name, Last Name, etc.)
columns_to_check = ["PRESCRIBER_ID"]
 
for column in columns_to_check:
    cursor.execute(f"""
        SELECT {column}, COUNT(*) 
        FROM bs_grp1_person_profile
        GROUP BY {column}
        HAVING COUNT(*) > 1;
    """)
    duplicates = cursor.fetchall()
    if duplicates:
        print(f"Warning: Duplicates found in {column} column!")
        for row in duplicates:
            print(f"  Duplicate Value: {row[0]} appears {row[1]} times.")

            
print("DQM CHECKS DONE FOR bs_grp1_prescriber")
# Commit the changes if any (rollback used to make sure no change to the table schema)
conn.rollback()

conn.commit()

DQM CHECKS DONE FOR bs_grp1_prescriber


Zip_to_Terr (bs_grp1_zip_terr)

In [209]:
# Check for Primary Key consistency (i.e., unique and non-null ZIP_CODE)
conn.rollback()

cursor.execute("""
    SELECT COUNT(*) 
    FROM bs_grp1_zip_terr
    WHERE ZIP_CODE IS NULL;
""")
null_count = cursor.fetchone()[0]
if null_count > 0:
    print(f"Warning: There are {null_count} NULL values in ZIP_CODE column!")

conn.rollback()

cursor.execute("""
    SELECT COUNT(DISTINCT ZIP_CODE) 
    FROM bs_grp1_zip_terr;
""")
distinct_count = cursor.fetchone()[0]
conn.rollback()

cursor.execute("""
    SELECT COUNT(ZIP_CODE) 
    FROM bs_grp1_zip_terr;
""")
total_count = cursor.fetchone()[0]

if distinct_count != total_count:
    print(f"Warning: There are duplicates in the ZIP_CODE column!")

# Check for duplicates in other columns (Territory_ID)
columns_to_check = ["ZIP_CODE", "Territory_ID"]

for column in columns_to_check:
    cursor.execute(f"""
        SELECT {column}, COUNT(*) 
        FROM bs_grp1_zip_terr
        GROUP BY {column}
        HAVING COUNT(*) > 1;
    """)
    duplicates = cursor.fetchall()
    if duplicates:
        print(f"Warning: Duplicates found in {column} column!")
        for row in duplicates:
            print(f"  Duplicate Value: {row[0]} appears {row[1]} times.")

print("DQM CHECKS DONE FOR bs_grp1_zip_terr")
# Commit the changes if any (rollback used to make sure no change to the table schema)
conn.rollback()

conn.commit()


  Duplicate Value: T-111100 appears 81 times.
  Duplicate Value: T-111200 appears 81 times.
  Duplicate Value: T-111300 appears 81 times.
  Duplicate Value: T-111400 appears 81 times.
  Duplicate Value: T-111500 appears 81 times.
  Duplicate Value: T-111600 appears 81 times.
  Duplicate Value: T-111700 appears 81 times.
  Duplicate Value: T-112100 appears 81 times.
  Duplicate Value: T-112200 appears 81 times.
  Duplicate Value: T-112300 appears 81 times.
  Duplicate Value: T-112400 appears 81 times.
  Duplicate Value: T-112500 appears 81 times.
  Duplicate Value: T-112600 appears 81 times.
  Duplicate Value: T-112700 appears 81 times.
  Duplicate Value: T-113100 appears 81 times.
  Duplicate Value: T-113200 appears 81 times.
  Duplicate Value: T-113300 appears 81 times.
  Duplicate Value: T-113400 appears 81 times.
  Duplicate Value: T-113500 appears 81 times.
  Duplicate Value: T-113600 appears 81 times.
  Duplicate Value: T-113700 appears 81 times.
  Duplicate Value: T-114100 appear

In [244]:
import psycopg2

def check_data_quality(conn, table_name, pk_columns, duplicate_columns=None):

    for pk_column in pk_columns:
        cursor.execute(f"""
            SELECT COUNT(*) 
            FROM {table_name} 
            WHERE {pk_column} IS NULL;
        """)
        null_count = cursor.fetchone()[0]
        if null_count > 0:
            print(f"Warning: There are {null_count} NULL values in {pk_column} column!")

    cursor.execute(f"""
        SELECT COUNT(DISTINCT {', '.join(pk_columns)}) 
        FROM {table_name};
    """)
    distinct_count = cursor.fetchone()[0]

    cursor.execute(f"""
        SELECT COUNT({', '.join(pk_columns)}) 
        FROM {table_name};
    """)
    total_count = cursor.fetchone()[0]

    if distinct_count != total_count:
        print(f"Warning: There are duplicates in {', '.join(pk_columns)} column(s)!")

    columns_to_check = duplicate_columns if duplicate_columns else pk_columns
    
    for column in columns_to_check:
        cursor.execute(f"""
            SELECT {column}, COUNT(*) 
            FROM {table_name}
            GROUP BY {column}
            HAVING COUNT(*) > 1;
        """)
        duplicates = cursor.fetchall()
        if duplicates:
            print(f"Warning: Duplicates found in {column} column!")
            for row in duplicates:
                print(f"  Duplicate Value: {row[0]} appears {row[1]} times.")

    print(f"DQM CHECKS DONE FOR {table_name}")
    
    conn.rollback()
    conn.commit()


check_data_quality(
    conn,
    'bs_grp1_person_profile',
    ['PRESCRIBER_ID'],  
    ['PRESCRIBER_ID']  
)
check_data_quality(
    conn,
    'bs_grp1_mkt_desc',
    ['mkt_cd'],  
    ['mkt_cd' , 'mkt_nm']  
)
check_data_quality(
    conn,
    'bs_grp1_prod_master',
    ['prd_pack_id'],
    ['prd_pack_id']
)
check_data_quality(
    conn,
    'bs_grp1_person_address',
    ['person_code'],
    ['person_code']
)

check_data_quality(
    conn,
    'bs_grp1_geo_hierarchy',
    ['geo_code'],
    ['geo_code']
)
check_data_quality(
    conn,
    'bs_grp1_zip_terr',
    ['ZIP_CODE'],
    ['ZIP_CODE']
)

DQM CHECKS DONE FOR bs_grp1_person_profile
DQM CHECKS DONE FOR bs_grp1_mkt_desc
DQM CHECKS DONE FOR bs_grp1_prod_master
DQM CHECKS DONE FOR bs_grp1_person_address
  Duplicate Value: D-111000 appears 2 times.
DQM CHECKS DONE FOR bs_grp1_geo_hierarchy
DQM CHECKS DONE FOR bs_grp1_zip_terr


In [263]:
import psycopg2

def check_composite_key_data_quality(conn, table_name, pk_columns, duplicate_columns=None):
    """
    This function performs data quality checks on the given table for:
    1. NULL values in primary key columns
    2. Duplicates in primary key or other specified columns
    
    :param conn: The connection object to the PostgreSQL database
    :param table_name: The name of the table to check
    :param pk_columns: A list of primary key columns (can be composite key)
    :param duplicate_columns: (Optional) List of columns to check for duplicates (defaults to pk_columns)
    """
    cursor = conn.cursor()
    
    # Ensure the transaction is not in an error state before starting
    try:
        # Step 1: Check for NULL values in primary key columns
        for pk_column in pk_columns:
            cursor.execute(f"""
                SELECT COUNT(*) 
                FROM {table_name} 
                WHERE {pk_column} IS NULL;
            """)
            null_count = cursor.fetchone()[0]
            if null_count > 0:
                print(f"Warning: There are {null_count} NULL values in {pk_column} column!")
        
        # Step 2: Check for duplicates in the combination of primary key columns
        cursor.execute(f"""
            SELECT COUNT(DISTINCT {', '.join(pk_columns)}) 
            FROM {table_name};
        """)
        distinct_count = cursor.fetchone()[0]

        cursor.execute(f"""
            SELECT COUNT({', '.join(pk_columns)}) 
            FROM {table_name};
        """)
        total_count = cursor.fetchone()[0]

        if distinct_count != total_count:
            print(f"Warning: There are duplicates in the composite primary key ({', '.join(pk_columns)})!")

        # Step 3: Check for duplicates in the specified columns (either pk_columns or other provided columns)
        columns_to_check = duplicate_columns if duplicate_columns else pk_columns
        
        for column in columns_to_check:
            cursor.execute(f"""
                SELECT {column}, COUNT(*) 
                FROM {table_name}
                GROUP BY {column}
                HAVING COUNT(*) > 1;
            """)
            duplicates = cursor.fetchall()
            if duplicates:
                print(f"Warning: Duplicates found in {column} column!")
                for row in duplicates:
                    print(f"  Duplicate Value: {row[0]} appears {row[1]} times.")
        
        print(f"DQM CHECKS DONE FOR {table_name}")
    
        # Commit the transaction after the checks
        conn.commit()

    except Exception as e:
        # Rollback if an error occurs
        print(f"Error occurred: {e}")
        conn.rollback()

# Call the function for the 'bs_grp1_sales' table with composite primary key columns
check_composite_key_data_quality(
    conn,
    'bs_grp1_sales',
    ['psbr_id', 'prod_id'],  # Composite primary key columns
    ['psbr_id', 'prod_id']   # Check duplicates based on the composite key
)


Error occurred: function count(character varying, character varying) does not exist
HINT:  No function matches the given name and argument types. You may need to add explicit type casts.



# DATA FROM BASE --> STAGE

Prescriber (stg_grp1_prescriber)

In [204]:
sql_copy = """
INSERT INTO stg_grp1_prescriber (PRESCRIBER_ID, FIRST_NAME, LAST_NAME, PREFFERED_NAME, GENDER, EMAIL, PERSON_SPECIALTY)
SELECT PRESCRIBER_ID, FIRST_NAME, LAST_NAME, PREFFERED_NAME, GENDER, EMAIL, PERSON_SPECIALTY
FROM bs_grp1_person_profile;
"""
conn.rollback()  # Ensure any existing transaction is rolled back
cursor.execute(sql_copy)  # Execute the insert query
conn.commit()  # Commit the changes
print("Data copied from bs_grp1_person_profile to stg_grp1_prescriber successfully!") 


Data copied from bs_grp1_person_profile to stg_grp1_prescriber successfully!


In [250]:
sql_copy = """
INSERT INTO stg_grp1_zip_terr (ZIP_CODE, Territory_ID)
SELECT ZIP_CODE, Territory_ID
FROM bs_grp1_zip_terr;
"""
conn.rollback()  # Ensure any existing transaction is rolled back
cursor.execute(sql_copy)  # Execute the insert query
conn.commit()  # Commit the changes
print("Data copied from bs_grp1_zip_terr to stg_grp1_zip_terr successfully!")


Data copied from bs_grp1_zip_terr to stg_grp1_zip_terr successfully!


In [251]:
# SQL query to copy data from bs_grp1_person_address to stg_grp1_prescriber_address
sql_copy = """
INSERT INTO stg_grp1_prescriber_address (PRESCRIBER_ID, ADDRESS_1, ADDRESS_2, CITY, STATE, ZIP_CODE)
SELECT PERSON_CODE, ADDRESS_1, ADDRESS_2, CITY, STATE, ZIP_CODE
FROM bs_grp1_person_address;
"""

# Rollback any active transaction to ensure no incomplete data
conn.rollback()

# Execute the insert query to move data
cursor.execute(sql_copy)

# Commit the changes to finalize the migration
conn.commit()

# Print success message
print("Data copied from bs_grp1_person_address to stg_grp1_prescriber_address successfully!")


Data copied from bs_grp1_person_address to stg_grp1_prescriber_address successfully!


In [252]:
# SQL query to copy data from bs_grp1_mkt_desc to stg_grp1_mkt_desc
sql_copy = """
INSERT INTO stg_grp1_mkt_desc (MKT_CD, MKT_NM)
SELECT MKT_CD, MKT_NM
FROM bs_grp1_mkt_desc;
"""

# Rollback any active transaction to ensure no incomplete data
conn.rollback()

# Execute the insert query to move data
cursor.execute(sql_copy)

# Commit the changes to finalize the migration
conn.commit()

# Print success message
print("Data copied from bs_grp1_mkt_desc to stg_grp1_mkt_desc successfully!")


Data copied from bs_grp1_mkt_desc to stg_grp1_mkt_desc successfully!


In [256]:
# SQL query to copy data from bs_grp1_prod_master to stg_grp1_prod_master
# SQL query to copy data from bs_grp1_prod_master to stg_grp1_prod_master
# SQL query to copy data from bs_grp1_prod_master to stg_grp1_prod_master with casting
sql_copy = """
INSERT INTO stg_grp1_prod_master (MKT_CD, PRD_PACK_ID, PRODUCT_NAME, STRENGTH, BRAND_ID, Brand_Name, PROD_MNF)
SELECT MARKET_CD, 
       CAST(PRD_PACK_ID AS INT),  
       PRODUCT_NAME, 
       IMS_STRENGTH_LEVEL, 
       CAST(BRAND_ID AS INT),
       BRAND_NAME, 
       PROD_MNF
FROM bs_grp1_prod_master;
"""



# Rollback any active transaction to ensure no incomplete data
conn.rollback()

# Execute the insert query to move data
cursor.execute(sql_copy)
# Commit the changes to finalize the migration
conn.commit()

# Print success message
print("Data copied from bs_grp1_prod_master to stg_grp1_prod_master successfully!")


Data copied from bs_grp1_prod_master to stg_grp1_prod_master successfully!


In [257]:
# SQL query to copy data from bs_grp1_geo_hierarchy to stg_grp1_geo_hierarchy
# Casting is applied where needed
sql_copy = """
INSERT INTO stg_grp1_geo_hierarchy (GEO_CODE, GEO_NAME, GEO_LEVEL, PARENT_ID)
SELECT GEO_CODE, 
       GEO_NAME, 
       GEO_LEVEL, 
       PARENT_ID
FROM bs_grp1_geo_hierarchy;
"""

# Rollback any active transaction to ensure no incomplete data
conn.rollback()

# Execute the insert query to move data
cursor.execute(sql_copy)

# Commit the changes to finalize the migration
conn.commit()

# Print success message
print("Data copied from bs_grp1_geo_hierarchy to stg_grp1_geo_hierarchy successfully!")


Data copied from bs_grp1_geo_hierarchy to stg_grp1_geo_hierarchy successfully!


In [258]:
# SQL query to copy data from bs_grp1_sales to stg_grp1_sales with casting
sql_copy = """
INSERT INTO stg_grp1_sales (
    PRESCRIBER_ID, 
    PRD_PACK_ID, 
    MONTH_1_NRX_UNIT, MONTH_2_NRX_UNIT, MONTH_3_NRX_UNIT, MONTH_4_NRX_UNIT, MONTH_5_NRX_UNIT, MONTH_6_NRX_UNIT, 
    MONTH_7_NRX_UNIT, MONTH_8_NRX_UNIT, MONTH_9_NRX_UNIT, MONTH_10_NRX_UNIT, MONTH_11_NRX_UNIT, MONTH_12_NRX_UNIT, 
    MONTH_1_NRX_VOL, MONTH_2_NRX_VOL, MONTH_3_NRX_VOL, MONTH_4_NRX_VOL, MONTH_5_NRX_VOL, MONTH_6_NRX_VOL, 
    MONTH_7_NRX_VOL, MONTH_8_NRX_VOL, MONTH_9_NRX_VOL, MONTH_10_NRX_VOL, MONTH_11_NRX_VOL, MONTH_12_NRX_VOL, 
    MONTH_1_NRX_DOLLAR, MONTH_2_NRX_DOLLAR, MONTH_3_NRX_DOLLAR, MONTH_4_NRX_DOLLAR, MONTH_5_NRX_DOLLAR, 
    MONTH_6_NRX_DOLLAR, MONTH_7_NRX_DOLLAR, MONTH_8_NRX_DOLLAR, MONTH_9_NRX_DOLLAR, MONTH_10_NRX_DOLLAR, 
    MONTH_11_NRX_DOLLAR, MONTH_12_NRX_DOLLAR, MONTH_1_RRX_UNIT, MONTH_2_RRX_UNIT, MONTH_3_RRX_UNIT, 
    MONTH_4_RRX_UNIT, MONTH_5_RRX_UNIT, MONTH_6_RRX_UNIT, MONTH_7_RRX_UNIT, MONTH_8_RRX_UNIT, MONTH_9_RRX_UNIT, 
    MONTH_10_RRX_UNIT, MONTH_11_RRX_UNIT, MONTH_12_RRX_UNIT, MONTH_1_RRX_VOL, MONTH_2_RRX_VOL, MONTH_3_RRX_VOL, 
    MONTH_4_RRX_VOL, MONTH_5_RRX_VOL, MONTH_6_RRX_VOL, MONTH_7_RRX_VOL, MONTH_8_RRX_VOL, MONTH_9_RRX_VOL, 
    MONTH_10_RRX_VOL, MONTH_11_RRX_VOL, MONTH_12_RRX_VOL, MONTH_1_RRX_DOLLAR, MONTH_2_RRX_DOLLAR, 
    MONTH_3_RRX_DOLLAR, MONTH_4_RRX_DOLLAR, MONTH_5_RRX_DOLLAR, MONTH_6_RRX_DOLLAR, MONTH_7_RRX_DOLLAR, 
    MONTH_8_RRX_DOLLAR, MONTH_9_RRX_DOLLAR, MONTH_10_RRX_DOLLAR, MONTH_11_RRX_DOLLAR, MONTH_12_RRX_DOLLAR, 
    MONTH_1_TRX_UNIT, MONTH_2_TRX_UNIT, MONTH_3_TRX_UNIT, MONTH_4_TRX_UNIT, MONTH_5_TRX_UNIT, MONTH_6_TRX_UNIT, 
    MONTH_7_TRX_UNIT, MONTH_8_TRX_UNIT, MONTH_9_TRX_UNIT, MONTH_10_TRX_UNIT, MONTH_11_TRX_UNIT, MONTH_12_TRX_UNIT, 
    MONTH_1_TRX_VOL, MONTH_2_TRX_VOL, MONTH_3_TRX_VOL, MONTH_4_TRX_VOL, MONTH_5_TRX_VOL, MONTH_6_TRX_VOL, 
    MONTH_7_TRX_VOL, MONTH_8_TRX_VOL, MONTH_9_TRX_VOL, MONTH_10_TRX_VOL, MONTH_11_TRX_VOL, MONTH_12_TRX_VOL, 
    MONTH_1_TRX_DOLLAR, MONTH_2_TRX_DOLLAR, MONTH_3_TRX_DOLLAR, MONTH_4_TRX_DOLLAR, MONTH_5_TRX_DOLLAR, 
    MONTH_6_TRX_DOLLAR, MONTH_7_TRX_DOLLAR, MONTH_8_TRX_DOLLAR, MONTH_9_TRX_DOLLAR, MONTH_10_TRX_DOLLAR, 
    MONTH_11_TRX_DOLLAR, MONTH_12_TRX_DOLLAR)
SELECT 
    PSBR_ID, 
    CAST(PROD_ID AS INT),  -- Cast PROD_ID as INT
    CAST(MONTH_1_NRX_UNIT AS INT), CAST(MONTH_2_NRX_UNIT AS INT), CAST(MONTH_3_NRX_UNIT AS INT), CAST(MONTH_4_NRX_UNIT AS INT),
    CAST(MONTH_5_NRX_UNIT AS INT), CAST(MONTH_6_NRX_UNIT AS INT), CAST(MONTH_7_NRX_UNIT AS INT), CAST(MONTH_8_NRX_UNIT AS INT),
    CAST(MONTH_9_NRX_UNIT AS INT), CAST(MONTH_10_NRX_UNIT AS INT), CAST(MONTH_11_NRX_UNIT AS INT), CAST(MONTH_12_NRX_UNIT AS INT),
    CAST(MONTH_1_NRX_VOL AS INT), CAST(MONTH_2_NRX_VOL AS INT), CAST(MONTH_3_NRX_VOL AS INT), CAST(MONTH_4_NRX_VOL AS INT),
    CAST(MONTH_5_NRX_VOL AS INT), CAST(MONTH_6_NRX_VOL AS INT), CAST(MONTH_7_NRX_VOL AS INT), CAST(MONTH_8_NRX_VOL AS INT),
    CAST(MONTH_9_NRX_VOL AS INT), CAST(MONTH_10_NRX_VOL AS INT), CAST(MONTH_11_NRX_VOL AS INT), CAST(MONTH_12_NRX_VOL AS INT),
    CAST(MONTH_1_NRX_DOLLAR AS INT), CAST(MONTH_2_NRX_DOLLAR AS INT), CAST(MONTH_3_NRX_DOLLAR AS INT), 
    CAST(MONTH_4_NRX_DOLLAR AS INT), CAST(MONTH_5_NRX_DOLLAR AS INT), CAST(MONTH_6_NRX_DOLLAR AS INT), 
    CAST(MONTH_7_NRX_DOLLAR AS INT), CAST(MONTH_8_NRX_DOLLAR AS INT), CAST(MONTH_9_NRX_DOLLAR AS INT), 
    CAST(MONTH_10_NRX_DOLLAR AS INT), CAST(MONTH_11_NRX_DOLLAR AS INT), CAST(MONTH_12_NRX_DOLLAR AS INT),
    CAST(MONTH_1_RRX_UNIT AS INT), CAST(MONTH_2_RRX_UNIT AS INT), CAST(MONTH_3_RRX_UNIT AS INT), 
    CAST(MONTH_4_RRX_UNIT AS INT), CAST(MONTH_5_RRX_UNIT AS INT), CAST(MONTH_6_RRX_UNIT AS INT),
    CAST(MONTH_7_RRX_UNIT AS INT), CAST(MONTH_8_RRX_UNIT AS INT), CAST(MONTH_9_RRX_UNIT AS INT), 
    CAST(MONTH_10_RRX_UNIT AS INT), CAST(MONTH_11_RRX_UNIT AS INT), CAST(MONTH_12_RRX_UNIT AS INT),
    CAST(MONTH_1_RRX_VOL AS INT), CAST(MONTH_2_RRX_VOL AS INT), CAST(MONTH_3_RRX_VOL AS INT),
    CAST(MONTH_4_RRX_VOL AS INT), CAST(MONTH_5_RRX_VOL AS INT), CAST(MONTH_6_RRX_VOL AS INT),
    CAST(MONTH_7_RRX_VOL AS INT), CAST(MONTH_8_RRX_VOL AS INT), CAST(MONTH_9_RRX_VOL AS INT),
    CAST(MONTH_10_RRX_VOL AS INT), CAST(MONTH_11_RRX_VOL AS INT), CAST(MONTH_12_RRX_VOL AS INT),
    CAST(MONTH_1_RRX_DOLLAR AS INT), CAST(MONTH_2_RRX_DOLLAR AS INT), CAST(MONTH_3_RRX_DOLLAR AS INT),
    CAST(MONTH_4_RRX_DOLLAR AS INT), CAST(MONTH_5_RRX_DOLLAR AS INT), CAST(MONTH_6_RRX_DOLLAR AS INT),
    CAST(MONTH_7_RRX_DOLLAR AS INT), CAST(MONTH_8_RRX_DOLLAR AS INT), CAST(MONTH_9_RRX_DOLLAR AS INT),
    CAST(MONTH_10_RRX_DOLLAR AS INT), CAST(MONTH_11_RRX_DOLLAR AS INT), CAST(MONTH_12_RRX_DOLLAR AS INT),
    CAST(MONTH_1_TRX_UNIT AS INT), CAST(MONTH_2_TRX_UNIT AS INT), CAST(MONTH_3_TRX_UNIT AS INT),
    CAST(MONTH_4_TRX_UNIT AS INT), CAST(MONTH_5_TRX_UNIT AS INT), CAST(MONTH_6_TRX_UNIT AS INT),
    CAST(MONTH_7_TRX_UNIT AS INT), CAST(MONTH_8_TRX_UNIT AS INT), CAST(MONTH_9_TRX_UNIT AS INT),
    CAST(MONTH_10_TRX_UNIT AS INT), CAST(MONTH_11_TRX_UNIT AS INT), CAST(MONTH_12_TRX_UNIT AS INT),
    CAST(MONTH_1_TRX_VOL AS INT), CAST(MONTH_2_TRX_VOL AS INT), CAST(MONTH_3_TRX_VOL AS INT),
    CAST(MONTH_4_TRX_VOL AS INT), CAST(MONTH_5_TRX_VOL AS INT), CAST(MONTH_6_TRX_VOL AS INT),
    CAST(MONTH_7_TRX_VOL AS INT), CAST(MONTH_8_TRX_VOL AS INT), CAST(MONTH_9_TRX_VOL AS INT),
    CAST(MONTH_10_TRX_VOL AS INT), CAST(MONTH_11_TRX_VOL AS INT), CAST(MONTH_12_TRX_VOL AS INT),
    CAST(MONTH_1_TRX_DOLLAR AS INT), CAST(MONTH_2_TRX_DOLLAR AS INT), CAST(MONTH_3_TRX_DOLLAR AS INT),
    CAST(MONTH_4_TRX_DOLLAR AS INT), CAST(MONTH_5_TRX_DOLLAR AS INT), CAST(MONTH_6_TRX_DOLLAR AS INT),
    CAST(MONTH_7_TRX_DOLLAR AS INT), CAST(MONTH_8_TRX_DOLLAR AS INT), CAST(MONTH_9_TRX_DOLLAR AS INT),
    CAST(MONTH_10_TRX_DOLLAR AS INT), CAST(MONTH_11_TRX_DOLLAR AS INT), CAST(MONTH_12_TRX_DOLLAR AS INT)
FROM bs_grp1_sales;
"""

# Rollback any active transaction to ensure no incomplete data
conn.rollback()

# Execute the insert query to move data
cursor.execute(sql_copy)

# Commit the changes to finalize the migration
conn.commit()

# Print success message
print("Data copied from bs_grp1_sales to stg_grp1_sales successfully!")


Data copied from bs_grp1_sales to stg_grp1_sales successfully!


# FACT AND DIM TABLE CREATION